In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import pandas as pd
import sklearn

In [3]:
data_path = Path("data").resolve()
train_path = data_path / "train_spam.csv"
test_path = data_path / "test_spam.csv"

In [4]:
train_raw = pd.read_csv(train_path)
test_raw = pd.read_csv(test_path)

In [5]:
train_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16278 entries, 0 to 16277
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text_type  16278 non-null  object
 1   text       16278 non-null  object
dtypes: object(2)
memory usage: 254.5+ KB
